# Geometry

Useful formulas and algorithms to solve 2D geometric problems.

Ussual issues with geometry related problems:
* Tricky edge cases.
* Floating point precision problems.
* Tedious coding.

In [1]:
from dataclasses import dataclass
from typing import List
import math
# numpy IS NOT part of the standard libraries

EPS = 1E-8

## Points and Lines


We will define a point as a pair $(x,y)$

In [2]:
@dataclass
class point:
    x: float
    y: float

### Operations with points

Algebra of a point:

In [3]:
def __add__(self, t):
    return point(self.x + t.x, self.y + t.y)
def __sub__(self, t):
    return point(self.x - t.x, self.y - t.y)
def dot(self, a):
    return self.x*a.x + self.y*a.y

Distance from a point to the origin:

In [4]:
def norm(self):
    return math.sqrt(self.dot(self))

Rotation:

In [5]:
def rotate(self, theta):
    return point(
        self.x*math.cos(theta) + self.y*math.sin(theta),
        self.x*math.sin(theta) - self.y*math.cos(theta),
    )

Angle between 3 points (using the class/calling point as the center point):

In [6]:
def angle(self, a, c):
    s1 = a - self
    d1 = s1.norm()

    s2 = c - self
    d2 = s2.norm()

    return math.acos(s1.dot(s2)/(d1*d2))

### Lines

A line can be defined as:
* A pair $(m,n)$ such that $$y=mx+n$$
* A triplet $(a,b,c)$ such that $$ax+by+c=0$$

In [7]:
@dataclass
class line:
    a: float
    b: float
    c: float

Given two points $(x_1,y_1)$ and $(x_2,y_2)$, the line passing through both points can be defined as: $$m = \frac{y_2-y_1}{x_2-x_1}$$ $$n = y_1 - mx_1 \text{ or } y_2 - m x_2$$

Or, for the general equation: $$a=1,b=0,c=-x_1$$ if $x_1 = x_2$ and $$a = -\frac{y_1-y_2}{x_1-x_2},b=1,c=-a x_1 - y_1$$ otherwise.

In [8]:
@staticmethod
def from_points(p1, p2):
    if abs(p1.x - p2.x) < EPS:
        return line(1.0, 0.0, -p1.x)
    else:
        a = -(p1.y - p2.y) / (p1.x - p2.x)
        b = 1.0
        c = -(a * p1.x) - p1.y
        return line(a, b, c)

We can discover the slope, or y crossing and x crossing as follows:

In [9]:
def slope(self):
    return -self.a / self.b
def y_cross(self):
    return -self.c / self.b
def x_cross(self):
    return -self.c / self.a

Lines can also be represented as: $$ x\hat n = d$$

Where $\hat{n}$ is the normal of the line and $d$ is the distance from the origin to the line:

In [10]:
def normal(self):
    return point(
        self.a / math.sqrt(self.a**2 + self.b**2),
        self.b / math.sqrt(self.a**2 + self.b**2)
    )
def d(self):
    return -self.c / math.sqrt(self.a**2 + self.b**2)

Given two lines $L_1=(a_1,b_1,c_1)$ and $L_2=(a_2,b_2,c_2)$, their interesection occurs at the point:
$$\left(\frac{-b_1c_2 + b_2c_1}{a_1b_2 - a_2b_1}, \frac{-
a_2c_1+a_1c_2}{a_1b_2 - a_2b_1}\right)$$

In [11]:
def intersect(self, l):
    return point(
        (self.b*l.c - l.b*self.c)/ (self.a*l.b - l.a*self.b),
        (self.a*l.c - l.a*self.c)/ (self.a*l.b - l.a*self.b)
    )

In [12]:
def are_parallel(self, line):
    return abs(
        (self.a*line.a* + self.b*line.b)
        / (math.sqrt(self.a**2 + self.b**2)*math.sqrt(line.a**2 + line.b**2))
    - 1.0) < EPS

The angle between two lines is: $$\alpha = \text{arctan}\left(\frac{m_1-m_2}
{1+m_1m_2}\right)$$

or for a general line: $$ \alpha = \text{arccos}\left(\frac{a_1a_2 + b_1b_2}{\sqrt{(a_1^2 +
b_1^2)(a_2^2 + b_2^2)}}\right) $$

In [13]:
def angle(self, line):
    return math.acos(
        (self.a*line.a + self.b*line.b)
        / (math.sqrt(self.a**2 + self.b**2)*math.sqrt(line.a**2+line.b**2))
    )

A perpendicular line to $L_1$ has a parameter: $$m_2=-\frac{1}{m_1}$$

## Triangles

Area of a triangle: $$ A = \frac{bh}{2} $$
Heron's formula: $$ A = \sqrt{s(s-a)(s-b)(s-c)}$$
with $s = \frac{a+b+c}{2}$ the semi-perimeter of the triangle.

The radius of the ***incircle*** of a triangle is given by:
$$r = \frac{A}{s}$$
with $A$ the area of the triangle and $s$ the semi-perimeter.

The center of the ***incircle*** is the intersection of triangle's angle bisectors

The radius of the circumcircle is given by the following expression:
$$R = \frac{abc}{4A}$$
where $a, b, c$ are the side lengths of the triangle and $A$ the area of the triangle.

The center of the circumcircle is the intersection of the triangle's perpendicular bisectors

Cosine law:
$$ a^2 = b^2 + c^2 - 2bc\cos\theta_A $$
$$ b^2 = a^2 + c^2 - 2ac\cos\theta_B $$
$$ c^2 = a^2 + b^2 - 2ab\cos\theta_C $$

Sine law: $$ \frac{\sin\theta_A}{a} = \frac{\sin\theta_B}{b} = \frac{\sin\theta_C}{c} $$

## Cross-Product and Area of a Parallelepiped

Given two points in the 2D space, we define the cross-product as:
$$ (x_1,y_1) \times (x_2,y_2) = x_1 y_2 - x_2 y_1 $$

In [14]:
# Inside the Point class
def cross(self, p):
    return self.x*p.y - p.x*self.y

Given a parallelepiped defined by its vertex $(0,0), (x_1, y_1), (x_2, y_2)$ and $(x_1+x_1,y_1+y_2)$, we can find its area using the cross product between $(x_1, y_1), (x_2, y_2)$.
$$ A = \left|v_1 \times v_2\right| = \left|x_1 y_2 - x_2 y_1\right| $$

Similarly, the area of a triangle with one vertex at the origin, $v_1$ and $v_2$ is:
$$ A = \frac{\left|v_1 \times v_2\right|}{2} $$

We can also infer the relative position of the vectors.
If: $$ v_1 \times v_2 > 0 $$ then $v_1$ is in the clock-wise direction of $v_2$.

## Intersection of Line Segments

Given two segments $p_1 p_2$ and $p_3 p_4$ we are interested in their intersection.

In [15]:
@dataclass
class segment:
    p: point
    q: point

It is easy to check that to see if the segments intersect, we require to check that the end points of the segments are on opposite sides of the other segment.

![](segment_intersection.png)

To this end we can use the (normal) product as follows:
$$ (\vec{p_1 p_3} \times \vec{p_1 p_2}) \cdot (\vec{p_1 p_4} \times \vec{p_1 p_2}) $$

If the expression is less than zero, it means that both cross product have different sign, hence the end points of the second line segments are at both sides of the first line segment.

Then we repeat the procedure using $p_3 p_4$ as the first line segment.

In [16]:
def does_intersect(self, seg2, *, include_p=False, include_q=False):
    cross1 = (seg2.q - self.p).cross(self.q - self.p)
    cross2 = (seg2.p - self.p).cross(self.q - self.p)
    cross3 = (self.q - seg2.p).cross(seg2.q - seg2.p)
    cross4 = (self.p - seg2.p).cross(seg2.q - seg2.p)
    return (
        (cross1 * cross2 < 0 or
            (include_p and math.fabs(cross2) < EPS)
            or (include_q and math.fabs(cross1) < EPS))
        and (cross3 * cross4 < 0
            or (include_p and math.fabs(cross4) < EPS)
            or (include_q and math.fabs(cross3) < EPS))
    )

In [22]:
seg1 = segment(point(0, 0), point(1, 1))
seg2 = segment(point(0, 1), point(1, 0))
does_intersect(seg1, seg2), seg1.does_intersect(seg2)

(True, True)

What do the `include_q` and `include_p` parameters do?

## Circles

* All points less than a distance $r$ from the center point $(a, b)$.
* The perimeter of a circle is given by $2\pi r$, where $r$ is the radius of the circle.
* The area of a circle is given by $\pi r^2$.

![](circles.png)

* Length of an arc $ar$ where $a$ is the inner angle (in radians).
* The area of the sector is $\frac{ar^2}{2}$
* Length of the chord (by using the law of cosines) is $\sqrt{2r^2(1 - \cos a)}$
* Area of the segment is the area of the sector minus the area of an icoseles triangle of side lenghs $(r, r, \sqrt{2r^2(1 - \cos a)})$.

## Polygons

A polygon is defined as a <u>***sorted***</u> list of vertex $(p_1, p_2, \dots, p_n)$.

In [30]:
from itertools import islice, cycle

@dataclass
class polygon:
    vertices: List[point]

    def shifted_vertices(self, shift=1):
        # v2, v3, ...., vN, v1
        yield from islice(cycle(self.vertices), shift, len(self.vertices) + shift)
    
    @property
    def segments(self):
        for v1, v2 in zip(self.vertices, self.shifted_vertices()):
            yield segment(v1, v2)

    @property
    def perimeter(self):
        return sum((v1 - v2).norm() for v1, v2 in zip(self.vertices, self.shifted_vertices()))

The (signed) area of a polygon can be calculated using the following formula:

$$ A = \frac{1}{2}\left[\begin{array}{cc} x_0 & y_0\\ x_1 & y_1\\ \dots\\ x_N &
y_N\end{array}\right] $$

where the $[\cdot]$ is a type of determinant as follows

$$ \left[\begin{array}{cc} x_0 & y_0\\ x_1 & y_1\\ \dots\\ x_N & y_N\end{array}\right] = x_0
y_1 - x_1 y_0 + x_1 y_2 - x_2 y_1 + \cdots = \sum_{i=0}^{N-1} x_i y_{i+1} - x_{i+1} y_i $$

In [31]:
def area(self):
    return 0.5*sum(p2.y*p1.x - p2.x*p1.y for p1, p2 in zip(self.vertices, self.shifted_vertices()))

We can check if the polygon is convex if the cross product between consecutive line segments has the same orientation (sign of the cross product).

In [34]:
def is_convex(self):
    clockwise = iter((p2 - p1).cross(p3 - p2) > 0
                     for p1, p2, p3 in zip(self.vertices,
                                           self.shifted_vertices(1),
                                           self.shifted_vertices(2)))
    first = next(clockwise)
    return all(first == x for x in clockwise)

### Point in a Polygon

Given a polygon and a point $q$, how to check if the point $q$ is inside or outside the polygon?

![](point_in_polygon_segment.png)

* Find a point outside the polygon $p$
* Cast a ray from $p$ to $q$
* Check for the intersection of the ray $pq$ with all the segments of the polygon
* If there is an odd number of intersection, $q$ is inside the polygon, otherwise it is outside.

In [37]:
def is_inside(self, q):
    p = min(self.vertices, key=lambda v: v.x) - point(1, 0)
    crosses = sum(1 if segment(p, q).does_intersect(s, include_p=True) else 0 for s in self.segments)
    return crosses % 2 == 1

A second approach is to sum the signed angles between the point and all the vertices of the polygon. If they sum up to $2\pi$ then the point is inside the polygon.

![](point_in_polygon_angles.png)

```Python
## PSEUDO CODE
def inside(self, p):
    if len(self.vertices) == 0:
        return False
    s = 0
    for s in self.segments:
        if ccw(s.p, p, s.q):
            s += angle(s.p, p, s.q)
        else:
            s -= angle(s.p, p, s.q)
    return abs(abs(s) - 2*math.pi) < EPS
```

### Cutting a Polygon

Given a line or segment:

* Classify the vertices of the polygon in different sides of the line.
* Find the intersection points of the line with the polygon.
* Create two new polygons using the points on one side of the line plus the intersection points and the points on the other side of the line and the intersection points.

In [39]:
def polygon_split(self, s):
    vertices1 = []
    vertices2 = []
    ds = s.p - s.q
    l = line.from_points(s.p, s.q)
    
    u = self.vertices[-1]
    side = ds.cross(u - s.q)
    for v in self.vertices:
        cross_prod = ds.cross(v - s.q)
        if cross_prod*side < 0: 
            p = line.from_points(u, v).intersect(l)
            vertices1.append(p)
            vertices2.append(p)
        if cross_prod <= 0:
            vertices1.append(v)
        if cross_prod >= 0:
            vertices2.append(v)
        side = cross_prod
        u = v
    return polygon(vertices1), polygon(vertices2)

## Convex Hull

Given a set of points, the convex hull is the minimum convex polygon encapsulating all points.

1. Select the left-most point $p=p_0$ and any other point $q$.
2. For all other points $r$:
    1. If $pq \times pr < 0$ then $q \leftarrow r$.
3. Add $pr$ to the convex hull.
4. Set $p \leftarrow r$, and repeat point 2 until point $p_0$ is reached again.

In [41]:
def hull(points):
    if len(points) < 3:
        return polygon(points)
    q = min(points, key=lambda v: v.x)
    p = point(q.x, q.y - 1)
    ch = [p, q]
    while True:
        p, q = ch[-2], ch[-1]
        u = max((v for v in points if v != p and v != q),
        key=lambda x: q.angle(p, x))
        if u in ch:
            break
        ch.append(u)
    return polygon(ch[1:])

![](convex_hull.png)

## Definitions

### Point Class

In [18]:
@dataclass
class point:
    x: float
    y: float

    def __add__(self, t):
        return point(self.x + t.x, self.y + t.y)
    def __sub__(self, t):
        return point(self.x - t.x, self.y - t.y)
    def dot(self, a):
        return self.x*a.x + self.y*a.y

    def norm(self):
        return math.sqrt(self.dot(self))
    
    def rotate(self, theta):
        return point(
            self.x*math.cos(theta) + self.y*math.sin(theta),
            self.x*math.sin(theta) - self.y*math.cos(theta),
        )
    
    def angle(self, a, c):
        s1 = a - self
        d1 = s1.norm()

        s2 = c - self
        d2 = s2.norm()

        return math.acos(s1.dot(s2)/(d1*d2))

    def cross(self, p):
        return self.x*p.y - p.x*self.y

### Line Class

In [20]:
@dataclass
class line:
    a: float
    b: float
    c: float

    @staticmethod
    def from_points(p1, p2):
        if abs(p1.x - p2.x) < EPS:
            return line(1.0, 0.0, -p1.x)
        else:
            a = -(p1.y - p2.y) / (p1.x - p2.x)
            b = 1.0
            c = -(a * p1.x) - p1.y
            return line(a, b, c)
        
    def slope(self):
        return -self.a / self.b
    def y_cross(self):
        return -self.c / self.b
    def x_cross(self):
        return -self.c / self.a
    
    def normal(self):
        return point(
            self.a / math.sqrt(self.a**2 + self.b**2),
            self.b / math.sqrt(self.a**2 + self.b**2)
        )
    def d(self):
        return -self.c / math.sqrt(self.a**2 + self.b**2)
    
    def intersect(self, l):
        return point(
            (self.b*l.c - l.b*self.c)/ (self.a*l.b - l.a*self.b),
            (self.a*l.c - l.a*self.c)/ (self.a*l.b - l.a*self.b)
        )
    
    def are_parallel(self, line):
        return abs(
            (self.a*line.a* + self.b*line.b)
            / (math.sqrt(self.a**2 + self.b**2)*math.sqrt(line.a**2 + line.b**2))
        - 1.0) < EPS
    
    def angle(self, line):
        return math.acos(
            (self.a*line.a + self.b*line.b)
            / (math.sqrt(self.a**2 + self.b**2)*math.sqrt(line.a**2+line.b**2))
        )

### Segment Class

In [21]:
@dataclass
class segment:
    p: point
    q: point

    def does_intersect(self, seg2, *, include_p=False, include_q=False):
        cross1 = (seg2.q - self.p).cross(self.q - self.p)
        cross2 = (seg2.p - self.p).cross(self.q - self.p)
        cross3 = (self.q - seg2.p).cross(seg2.q - seg2.p)
        cross4 = (self.p - seg2.p).cross(seg2.q - seg2.p)
        return (
            (cross1 * cross2 < 0 or
                (include_p and math.fabs(cross2) < EPS)
                or (include_q and math.fabs(cross1) < EPS))
            and (cross3 * cross4 < 0
                or (include_p and math.fabs(cross4) < EPS)
                or (include_q and math.fabs(cross3) < EPS))
        )

### Polygon Class

In [40]:
from itertools import islice, cycle

@dataclass
class polygon:
    vertices: List[point]

    def shifted_vertices(self, shift=1):
        # v2, v3, ...., vN, v1
        yield from islice(cycle(self.vertices), shift, len(self.vertices) + shift)
    
    @property
    def segments(self):
        for v1, v2 in zip(self.vertices, self.shifted_vertices()):
            yield segment(v1, v2)

    @property
    def perimeter(self):
        return sum((v1 - v2).norm() for v1, v2 in zip(self.vertices, self.shifted_vertices()))
    
    @property
    def area(self):
        return 0.5*sum(p2.y*p1.x - p2.x*p1.y for p1, p2 in zip(self.vertices, self.shifted_vertices()))
    
    @property
    def is_convex(self):
        clockwise = iter((p2 - p1).cross(p3 - p2) > 0
                        for p1, p2, p3 in zip(self.vertices,
                                            self.shifted_vertices(1),
                                            self.shifted_vertices(2)))
        first = next(clockwise)
        return all(first == x for x in clockwise)
    
    def is_inside(self, q):
        p = min(self.vertices, key=lambda v: v.x) - point(1, 0)
        crosses = sum(1 if segment(p, q).does_intersect(s, include_p=True) else 0 for s in self.segments)
        return crosses % 2 == 1
    
    def polygon_split(self, s):
        vertices1 = []
        vertices2 = []
        ds = s.p - s.q
        l = line.from_points(s.p, s.q)
        
        u = self.vertices[-1]
        side = ds.cross(u - s.q)
        for v in self.vertices:
            cross_prod = ds.cross(v - s.q)
            if cross_prod*side < 0: 
                p = line.from_points(u, v).intersect(l)
                vertices1.append(p)
                vertices2.append(p)
            if cross_prod <= 0:
                vertices1.append(v)
            if cross_prod >= 0:
                vertices2.append(v)
            side = cross_prod
            u = v
        return polygon(vertices1), polygon(vertices2)

### Convex Hull Function

In [42]:
def hull(points):
    if len(points) < 3:
        return polygon(points)
    q = min(points, key=lambda v: v.x)
    p = point(q.x, q.y - 1)
    ch = [p, q]
    while True:
        p, q = ch[-2], ch[-1]
        u = max((v for v in points if v != p and v != q),
        key=lambda x: q.angle(p, x))
        if u in ch:
            break
        ch.append(u)
    return polygon(ch[1:])